In [1]:
import argparse
import math
from intern.remote.boss import BossRemote
from intern.resource.boss.resource import *
import configparser
#import grequests # for async requests, conflicts with requests somehow
import requests
import numpy as np
from numpy import genfromtxt
import h5py
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import shutil
import blosc
from IPython.core.debugger import set_trace
import sys
import os
import itertools
from functools import partial
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
from multiprocessing import cpu_count
import csv
import datetime
import morton


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def F0(cubes):
    f0 = [[np.sum(cubes[i,j,:,:,:]) for j in range(cubes.shape[1])] for i in range(cubes.shape[0])]
    #f0 = np.sum(cube)
    return(f0)
    
def F1(cubes):
    s = cubes.shape

    f1 = [[np.sum(cubes[i,j,:,:,:]) for j in range(cubes.shape[1])] for i in range(cubes.shape[0])]
    out = f1

    return(out)



In [3]:
def getCube(di):
    data = di['rem'].get_cutout(di['ch_rsc'], di['res'], di['xrng'],
          di['yrng'] ,di['zrng'])
    print(di['loc']) #DEBUG
    sys.stdout.flush() #DEBUG
    ### OUTPUT will be a numpy array in Z Y X order 
    avg = np.mean(data)
    return(avg)


In [4]:
CONFIG_FILE = "neurodata.conf"
COLL_NAME = "collman"
EXP_NAME = "M247514_Rorb_1_Site3Align2_LENS_Session1_CROP"
COORD_FRAME = COLL_NAME + "_" + EXP_NAME
BF = 5

LOCATIONS = "rorb_locations_at_buffered.csv"

CHAN_NAMES = ['DAPI1', 'DAPI2', 'DAPI3', 'GABA', 'GAD2', 'Gephyrin', 'GluN1',
                'MBP', 'PSD95', 'synapsin', 'TdTomato', 'VGlut1']

num_threads = 4

In [5]:
config = configparser.ConfigParser()
config.read(CONFIG_FILE)
TOKEN = config['Default']['token']
boss_url = ''.join( ( config['Default']['protocol'],'://',config['Default']['host'],'/v1/' ) )
#print(boss_url)
#'https://api.boss.neurodata.io/v1/'

#intern
rem = BossRemote(CONFIG_FILE)

In [7]:
L = genfromtxt(LOCATIONS,  delimiter=',', skip_header = 1, dtype='int32').tolist()
L

[[230, 186, 38, 0],
 [238, 188, 40, 1],
 [239, 209, 25, 4],
 [240, 241, 10, 5],
 [248, 228, 16, 6],
 [249, 220, 10, 7],
 [256, 218, 17, 8],
 [228, 183, 38, 9],
 [222, 181, 40, 10],
 [221, 184, 34, 11],
 [187, 192, 34, 12],
 [191, 190, 32, 13],
 [199, 193, 34, 14],
 [203, 194, 22, 15],
 [206, 179, 20, 16],
 [205, 188, 15, 17],
 [210, 188, 19, 18],
 [197, 193, 20, 19],
 [196, 194, 17, 20],
 [194, 182, 18, 21],
 [200, 187, 21, 22],
 [222, 183, 23, 23],
 [230, 177, 32, 24],
 [225, 172, 35, 25],
 [222, 163, 38, 26],
 [207, 164, 40, 27],
 [201, 163, 18, 29],
 [208, 163, 34, 31],
 [217, 189, 13, 34],
 [231, 196, 10, 35],
 [225, 178, 14, 36],
 [227, 171, 17, 37],
 [234, 174, 19, 38],
 [235, 167, 16, 39],
 [233, 154, 30, 40],
 [235, 156, 31, 41],
 [241, 169, 16, 42],
 [215, 164, 36, 44],
 [219, 156, 26, 45],
 [233, 160, 29, 46],
 [228, 156, 20, 47],
 [222, 155, 13, 49],
 [223, 161, 21, 50],
 [226, 169, 32, 51],
 [220, 178, 28, 52],
 [213, 184, 12, 53],
 [207, 183, 9, 56],
 [207, 180, 6, 57],
 [

In [8]:
L = genfromtxt(LOCATIONS,  delimiter=',', skip_header = 1, dtype='int32').tolist()
m = morton.Morton(dimensions=3, bits=64)
Lzo = [m.pack(L[i][0], L[i][1], L[i][2]) for i in range(len(L))]
#Lzo = sorted([m.pack(L[i][0], L[i][1], L[i][2]) for i in range(len(L))])
#                                                                                               
ind = np.argsort(Lzo)
loc = [m.unpack(l) for l in sorted(Lzo)]
                                                                                               
x_rng = [[x[0]-BF, x[0]+BF+1] for x in loc] 
y_rng = [[y[1]-BF, y[1]+BF+1] for y in loc] 
z_rng = [[z[2]-BF, z[2]+BF+1] for z in loc]
ids   = [L[i][3] for i in ind]


In [19]:
ChanList = []
for ch in CHAN_NAMES:
    di = [{
          'rem': rem,
          'ch_rsc':
            ChannelResource(ch,COLL_NAME,EXP_NAME,'image',datatype='uint8'),
          'ch'  : ch,
          'res' : 0,
          'loc' : loc[i],
          'xrng': x_rng[i],  
          'yrng': y_rng[i],  
          'zrng': z_rng[i],  
          'bf'  : BF 
         } for i in range(len(loc))]

    with ThreadPool(num_threads) as tp:
        out = tp.map(getCube, di)
        print(ch) ##DEBUG
        sys.stdout.flush() #DEBUG

    ChanList.append(np.asarray(out))

outArray = np.asarray(ChanList)
## outArray will be a numpy array in [channel, synapse, z, y, x] order
## this is C-ordering


[215, 174, 39]
[192, 211, 25]
[220, 178, 28]
[187, 166, 12]
[220, 160, 41]
[197, 213, 24]
[190, 181, 22]
[213, 185, 31]
[218, 169, 42]
[195, 218, 24]
[188, 155, 39]
[225, 178, 14]
[200, 189, 41]
[209, 212, 18]
[191, 161, 43]
[234, 183, 9]
[215, 181, 35]
[188, 171, 40]
[220, 213, 18]
[236, 178, 14]
[222, 178, 33]
[191, 190, 32]
[218, 216, 22]
[234, 189, 12]
[221, 184, 34]
[190, 183, 42]
[220, 211, 29]
[237, 191, 12]
[223, 189, 34]
[196, 155, 6]
[231, 196, 10]
[218, 184, 36]
[197, 157, 15]
[237, 188, 14]
[236, 196, 15]
[222, 181, 40]
[222, 155, 13][254, 178, 6]

[227, 207, 11]
[237, 166, 34]
[201, 156, 22]
[235, 167, 16]
[244, 203, 14]
[226, 169, 32]
[205, 158, 27]
[227, 171, 17]
[250, 207, 10]
[225, 172, 35]
[219, 156, 26]
[234, 174, 19]
[227, 215, 12]
[231, 173, 37]
[236, 159, 14]
[227, 163, 26]
[249, 220, 10]
[237, 161, 40]
[242, 157, 14]
[238, 195, 17]
[252, 171, 32]
[228, 156, 20]
[231, 163, 25]
[236, 199, 23]
[247, 167, 42]
[233, 154, 30]
[231, 195, 24]
[230, 164, 28]
[230, 177, 32

[275, 251, 38]
[293, 220, 26]
[270, 203, 17]
[295, 224, 17]
[308, 208, 18]
[277, 251, 36]
[257, 197, 28]
[292, 226, 30]
[315, 209, 22]
[281, 248, 42]
[276, 193, 20]
[297, 225, 27]
[305, 223, 17]
[289, 239, 32]
[274, 194, 28]
[312, 226, 17]
[311, 216, 20]
[307, 226, 24]
[283, 196, 30]
[309, 208, 30]
[268, 213, 21]
[300, 232, 35]
[309, 227, 29]
[317, 219, 25]
[256, 218, 17]
[303, 239, 36]
[306, 234, 29]
[315, 221, 30]
[262, 214, 26]
[292, 228, 40]
[313, 237, 24]
[264, 231, 6]
[263, 212, 30]
[271, 228, 7]
[266, 221, 28]
[292, 232, 41]
[312, 233, 30]
[260, 230, 10]
[191, 268, 18]
[288, 245, 16]
[308, 228, 33]
[225, 270, 29]
[290, 241, 22]
[311, 231, 38]
[191, 274, 29]
[315, 230, 40]
[240, 257, 20]
[298, 255, 16]
[188, 279, 31]
[299, 240, 34]
[252, 262, 18]
[294, 243, 28]
[188, 256, 32]
[314, 245, 34]
[242, 260, 26]
[293, 249, 26]
[191, 285, 37]
[315, 241, 37]
[302, 248, 24]
[242, 263, 31]
[197, 261, 14]
[275, 260, 6]
[297, 263, 25]
[240, 268, 25]
[198, 266, 8]
[285, 259, 6]
[288, 271, 30]


[282, 165, 22]
[223, 235, 36]
[286, 165, 20]
[244, 251, 10]
[293, 179, 17]
[282, 175, 22]
[199, 251, 32]
[255, 248, 10]
[289, 182, 16]
[284, 163, 26]
[210, 247, 35]
[226, 226, 19]
[290, 187, 16]
[217, 240, 37]
[233, 236, 26]
[283, 162, 30]
[296, 183, 25]
[216, 250, 34]
[307, 188, 20]
[248, 228, 16]
[278, 173, 29]
[223, 247, 40]
[254, 224, 23]
[307, 182, 31]
[211, 249, 40]
[258, 178, 16]
[245, 230, 25]
[312, 190, 28]
[208, 249, 42]
[258, 182, 16]
[247, 234, 24]
[215, 254, 43]
[278, 158, 34]
[236, 251, 19]
[264, 180, 18]
[217, 252, 40]
[284, 155, 36]
[269, 177, 22]
[254, 241, 28]
[218, 253, 43]
[266, 182, 21]
[229, 225, 39]
[199, 193, 34]
[277, 157, 41]
[256, 186, 18]
[231, 236, 34]
[195, 198, 34]
[311, 159, 34]
[262, 185, 18]
[200, 198, 33]
[237, 239, 32]
[315, 159, 33]
[245, 238, 40]
[204, 193, 36]
[256, 191, 16]
[304, 155, 40]
[198, 196, 41]
[264, 188, 22][236, 244, 35]

[256, 160, 34]
[230, 248, 39]
[259, 182, 30]
[261, 167, 33]
[226, 252, 38]
[222, 198, 38]
[264, 176, 28]
[265, 166,

[225, 176, 39]
[228, 156, 20]
[231, 195, 24]
[228, 183, 38]
[230, 164, 28]
[233, 154, 30]
[242, 196, 18]
[230, 186, 38]
[232, 166, 26]
[235, 156, 31]
[238, 188, 40]
[249, 195, 19]
[233, 160, 29]
[251, 159, 29]
[248, 193, 21]
[192, 162, 7]
[247, 176, 36]
[227, 172, 30]
[193, 171, 13]
[245, 207, 17]
[251, 165, 19]
[250, 179, 32]
[206, 175, 14]
[244, 204, 19]
[241, 169, 16]
[241, 185, 36]
[241, 194, 24]
[209, 168, 11]
[244, 175, 16]
[254, 184, 32]
[247, 206, 24]
[207, 180, 6]
[245, 170, 28]
[244, 184, 40]
[203, 182, 10]
[251, 203, 30]
[189, 207, 8]
[231, 176, 18]
[207, 183, 9]
[231, 214, 18]
[189, 205, 22]
[237, 188, 18]
[193, 188, 12]
[238, 215, 18]
[191, 207, 25]
[235, 179, 29]
[207, 186, 10]
[224, 223, 16]
[227, 190, 26]
[189, 208, 16]
[202, 190, 10]
[229, 220, 18]
[190, 245, 28]
[229, 186, 30]
[203, 187, 12]
[231, 217, 22]
[230, 191, 29]
[187, 192, 34]
[231, 208, 25]
[253, 177, 24]
[191, 217, 36]
[239, 209, 25]
[250, 188, 25]
[205, 188, 15]
[236, 217, 26]
[191, 231, 36]
[213, 184, 12]

[242, 263, 31]
[191, 285, 37]
[257, 283, 6]
[297, 263, 25]
[240, 268, 25]
[197, 261, 14]
[271, 274, 10]
[288, 271, 30]
[231, 274, 16]
[300, 266, 25]
[263, 284, 11]
[198, 266, 8]
[309, 263, 20]
[279, 276, 11]
[220, 261, 13]
[313, 256, 18]
[225, 277, 19]
[285, 276, 13]
[218, 268, 12]
[310, 267, 21]
[286, 278, 14]
[194, 275, 6]
[317, 264, 20]
[276, 281, 12]
[234, 277, 16]
[200, 276, 14]
[309, 262, 28]
[227, 285, 19]
[197, 281, 9]
[280, 284, 11]
[304, 270, 31]
[224, 285, 22]
[219, 286, 8]
[261, 257, 20]
[300, 281, 16]
[232, 277, 31]
[194, 262, 22]
[292, 281, 24]
[267, 260, 17]
[231, 282, 24]
[193, 266, 23]
[292, 286, 24]
[264, 256, 20]
[243, 279, 23]
[200, 269, 22]
[298, 283, 24]
[267, 263, 20]
[247, 284, 16]
[202, 258, 28]
[310, 275, 16]
[258, 268, 17]
[250, 278, 26]
[310, 272, 18]
[198, 267, 31]
[263, 259, 30]
[253, 275, 30]
[262, 261, 36]
[221, 259, 16]
[266, 257, 26]
[265, 269, 38]
[220, 261, 21]
[255, 283, 27]
[268, 256, 30]
[258, 262, 42]
[220, 271, 20]
[249, 284, 31]
[257, 270, 30]


...

[213, 185, 31]
[218, 169, 42]
[195, 218, 24]
[188, 155, 39]
[225, 178, 14]
[200, 189, 41]
[191, 161, 43]
[234, 183, 9]
[215, 181, 35]
[188, 171, 40]
[209, 212, 18][222, 178, 33]

[236, 178, 14]
[234, 189, 12]
[191, 190, 32]
[220, 213, 18]
[221, 184, 34]
[190, 183, 42]
[237, 191, 12]
[218, 216, 22]
[223, 189, 34]
[196, 155, 6]
[220, 211, 29][237, 188, 14]

[218, 184, 36]
[197, 157, 15]
[231, 196, 10]
[222, 155, 13]
[236, 196, 15]
[254, 178, 6]
[222, 181, 40]
[201, 156, 22]
[227, 207, 11]
[235, 167, 16]
[237, 166, 34]
[205, 158, 27]
[244, 203, 14]
[226, 169, 32]
[219, 156, 26]
[250, 207, 10]
[225, 172, 35]
[236, 159, 14]
[227, 215, 12]
[231, 173, 37]
[242, 157, 14]
[227, 171, 17]
[249, 220, 10]
[237, 161, 40]
[228, 156, 20]
[238, 195, 17]
[234, 174, 19]
[252, 171, 32]
[233, 154, 30]
[227, 163, 26]
[236, 199, 23]
[235, 156, 31]
[231, 163, 25]
[231, 195, 24]
[251, 159, 29]
[247, 167, 42]
[230, 164, 28]
[192, 162, 7]
[230, 177, 32]
[193, 171, 13]
[242, 196, 18]
[225, 176, 39]
[206, 175, 14]

[305, 223, 17]
[272, 230, 43]
[262, 214, 26]
[297, 225, 27]
[274, 236, 43]
[265, 250, 37]
[311, 216, 20]
[263, 212, 30]
[312, 226, 17]
[309, 208, 30]
[274, 242, 36]
[317, 219, 25]
[276, 245, 36]
[307, 226, 24]
[315, 221, 30]
[275, 251, 38]
[264, 231, 6]
[309, 227, 29]
[266, 221, 28]
[277, 251, 36]
[271, 228, 7]
[306, 234, 29]
[191, 268, 18]
[260, 230, 10]
[281, 248, 42]
[313, 237, 24]
[191, 274, 29]
[312, 233, 30][289, 239, 32]

[188, 279, 31]
[300, 232, 35]
[288, 245, 16]
[225, 270, 29]
[188, 256, 32]
[290, 241, 22]
[303, 239, 36]
[240, 257, 20]
[191, 285, 37]
[292, 228, 40]
[298, 255, 16]
[252, 262, 18]
[197, 261, 14]
[292, 232, 41]
[294, 243, 28]
[242, 260, 26]
[198, 266, 8]
[308, 228, 33][293, 249, 26]

[220, 261, 13]
[302, 248, 24]
[311, 231, 38]
[218, 268, 12]
[242, 263, 31]
[275, 260, 6]
[315, 230, 40]
[240, 268, 25]
[285, 259, 6]
[274, 258, 8]
[231, 274, 16]
[194, 275, 6]
[299, 240, 34]
[225, 277, 19]
[200, 276, 14]
[314, 245, 34]
[234, 277, 16]
[283, 268, 9]
[197, 281, 9]
[315

[313, 172, 27]
[247, 234, 24]
[269, 172, 19]
[211, 249, 40]
[293, 179, 17]
[236, 251, 19]
[265, 161, 28]
[208, 249, 42]
[289, 182, 16]
[254, 241, 28]
[258, 171, 28]
[290, 187, 16]
[199, 193, 34]
[283, 161, 17]
[296, 183, 25]
[215, 254, 43]
[195, 198, 34]
[283, 166, 19]
[217, 252, 40]
[307, 188, 20]
[200, 198, 33][282, 166, 21]

[218, 253, 43]
[307, 182, 31]
[282, 165, 22]
[229, 225, 39]
[312, 190, 28]
[286, 165, 20]
[231, 236, 34]
[278, 158, 34]
[204, 193, 36]
[282, 175, 22]
[237, 239, 32]
[198, 196, 41]
[284, 155, 36]
[284, 163, 26]
[245, 238, 40]
[222, 198, 38]
[277, 157, 41]
[283, 162, 30]
[199, 219, 37]
[278, 173, 29]
[311, 159, 34]
[198, 216, 39]
[236, 244, 35]
[258, 178, 16]
[198, 218, 40]
[315, 159, 33]
[230, 248, 39]
[213, 220, 38]
[258, 182, 16]
[304, 155, 40]
[215, 217, 40]
[264, 180, 18]
[256, 160, 34]
[226, 192, 34]
[269, 177, 22]
[261, 167, 33]
[266, 182, 21]
[226, 198, 36]
[265, 166, 35]
[230, 204, 38]
[256, 186, 18]
[259, 174, 38]
[226, 252, 38]
[262, 191, 40]
[257, 161,

[245, 207, 17]
[231, 163, 25]
[192, 162, 7]
[244, 204, 19]
[230, 177, 32]
[230, 164, 28]
[225, 176, 39]
[232, 166, 26]
[228, 183, 38]
[193, 171, 13]
[241, 194, 24]
[233, 160, 29]
[230, 186, 38]
[247, 206, 24]
[206, 175, 14]
[238, 188, 40]
[251, 203, 30]
[209, 168, 11]
[207, 180, 6]
[231, 214, 18]
[203, 182, 10]
[238, 215, 18]
[247, 176, 36]
[207, 183, 9]
[227, 172, 30][224, 223, 16]

[250, 179, 32]
[193, 188, 12]
[229, 220, 18]
[251, 165, 19]
[241, 185, 36]
[231, 217, 22]
[207, 186, 10]
[241, 169, 16]
[202, 190, 10]
[231, 208, 25]
[254, 184, 32]
[244, 175, 16]
[203, 187, 12]
[239, 209, 25]
[244, 184, 40]
[245, 170, 28]
[205, 188, 15]
[236, 217, 26]
[189, 207, 8]
[231, 176, 18]
[213, 184, 12]
[247, 212, 17]
[237, 188, 18]
[247, 219, 23]
[217, 189, 13]
[235, 179, 29]
[189, 205, 22]
[251, 219, 23]
[201, 163, 18]
[191, 207, 25]
[242, 215, 26]
[200, 174, 17]
[250, 208, 24]
[206, 172, 22]
[249, 215, 24]
[189, 208, 16]
[190, 245, 28]
[227, 190, 26]
[195, 227, 7]
[204, 232, 12]
[187, 192, 34]


[315, 241, 37]
[298, 255, 16]
[231, 282, 24]
[275, 260, 6]
[219, 286, 8]
[294, 243, 28]
[243, 279, 23]
[194, 262, 22]
[193, 266, 23]
[247, 284, 16]
[285, 259, 6]
[200, 269, 22]
[250, 278, 26]
[202, 258, 28]
[274, 258, 8]
[198, 267, 31]
[283, 268, 9]
[253, 275, 30]
[271, 278, 7]
[255, 283, 27]
[257, 283, 6]
[221, 259, 16]
[249, 284, 31]
[271, 274, 10]
[220, 261, 21]
[207, 270, 34]
[220, 271, 20]
[202, 264, 38]
[221, 268, 22]
[263, 284, 11]
[199, 260, 43]
[293, 249, 26]
[211, 263, 26]
[279, 276, 11]
[206, 263, 42]
[302, 248, 24]
[222, 262, 27]
[285, 276, 13]
[297, 263, 25]
[221, 267, 28]
[288, 271, 30][210, 263, 33]

[195, 276, 17]
[300, 266, 25]
[286, 278, 14]
[216, 256, 34]
[201, 273, 22]
[309, 263, 20]
[194, 283, 19]
[221, 258, 36]
[276, 281, 12]
[313, 256, 18]
[195, 280, 21]
[280, 284, 11]
[310, 267, 21]
[204, 286, 16]
[261, 257, 20]
[213, 265, 38]
[317, 264, 20]
[196, 279, 29]
[267, 260, 17]
[309, 262, 28]
[202, 273, 26]
[219, 265, 36]
[264, 256, 20]
[202, 273, 31]
[304, 270, 31]
[2

In [28]:
outArray.shape

(12, 793)

In [38]:
outArray

array([[  2.13974455,   2.46731781,   2.34861007, ...,   2.50488355,
          2.8121713 ,   2.26446281],
       [  5.13824192,   5.78963186,   4.06235913, ...,   3.86326071,
          5.040571  ,   3.54470323],
       [  3.11870774,   4.89406461,   3.91059354, ...,   3.62133734,
          3.81141998,   3.36739294],
       ..., 
       [  2.82419234,   6.77310293,   2.23591285, ...,   5.04733283,
          7.41172051,   4.49361382],
       [ 40.78587528,  30.16528926,   7.87678437, ...,  11.90608565,
         30.93087904,  13.29676935],
       [  7.20435763,  11.31930879,   7.17806161, ...,   8.47783621,
          8.64162284,  10.07438017]])

In [29]:
#np.savetxt("rorb_avg_at.csv", np.column_stack([ids, outArray]), delimiter=",", fmt = "%d", header = "ids, ")
#np.concatenate((trmp, outArray), axis = 2)

In [34]:
#np.hstack((np.transpose(np.asarray(ids)),outArray))

In [39]:
head = "DAPI1, DAPI2, DAPI3, GABA, GAD2, Gephyrin, GluN1, MBP, PSD95, synapsin, TdTomato, VGlut1"
np.savetxt("rorb_avg_at.csv", np.transpose(outArray), delimiter=",", fmt = "%f", header = head)


In [41]:
np.savetxt("rorb_avg_at_orderIDS.csv", np.transpose(ids), delimiter=",", fmt = "%d", header = 'ids')

In [46]:
np.savetxt("rorb_avg_at_orderLocations.csv", loc, delimiter=",", fmt = "%d", header = 'x,y,z')

In [12]:
[idss + 1 for idss in ids]

[71,
 64,
 104,
 101,
 73,
 14,
 174,
 97,
 98,
 50,
 96,
 95,
 46,
 1009,
 996,
 48,
 41,
 42,
 820,
 107,
 69,
 67,
 92,
 58,
 66,
 57,
 65,
 988,
 60,
 167,
 18,
 54,
 35,
 30,
 76,
 1007,
 90,
 51,
 173,
 89,
 22,
 17,
 985,
 23,
 74,
 1008,
 75,
 168,
 24,
 19,
 53,
 169,
 37,
 981,
 203,
 206,
 205,
 204,
 975,
 40,
 38,
 39,
 88,
 994,
 993,
 992,
 47,
 991,
 821,
 43,
 170,
 997,
 187,
 307,
 165,
 177,
 179,
 175,
 827,
 810,
 99,
 100,
 103,
 102,
 78,
 82,
 94,
 186,
 208,
 185,
 822,
 72,
 68,
 77,
 70,
 28,
 32,
 45,
 27,
 85,
 84,
 79,
 198,
 987,
 197,
 196,
 12,
 181,
 182,
 11,
 202,
 52,
 26,
 183,
 201,
 823,
 998,
 25,
 195,
 10,
 1,
 2,
 999,
 829,
 180,
 1001,
 171,
 148,
 146,
 145,
 147,
 272,
 13,
 122,
 129,
 1023,
 61,
 128,
 978,
 59,
 149,
 150,
 142,
 140,
 115,
 21,
 20,
 16,
 127,
 139,
 143,
 144,
 117,
 288,
 119,
 118,
 124,
 36,
 163,
 299,
 974,
 973,
 300,
 8,
 161,
 305,
 176,
 162,
 979,
 980,
 960,
 303,
 1012,
 959,
 969,
 301,
 289,
 294,
 296